### Notebook to analyze IFWASTE-Simulation results

In [ ]:
import os 
import pandas as pd 
import json
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import dask.dataframe as dd

In [ ]:
PATH = "/blue/carpena/haasehelen/ifwaste/data/"
CONFIG_PATH = "/blue/carpena/haasehelen/ifwaste/model/config.json"

### Loading necessary parameters from config file
- weights per serving of each food category

In [13]:
FGMEAT_KG = FGDAIRY_KG = FGBAKED_KG = FGVEGETABLE_KG = FGDRYFOOD_KG = FGSNACKS_KG = FGSTOREPREPARED_KG = None
with open(CONFIG_PATH) as f:
    config = json.load(f)
    FGMEAT_KG = config["Food"]["FGMeat"]["kg_per_serving"]
    FGDAIRY_KG = config["Food"]["FGDairy"]["kg_per_serving"]
    FGBAKED_KG = config["Food"]["FGBaked"]["kg_per_serving"]
    FGVEGETABLE_KG = config["Food"]["FGVegetable"]["kg_per_serving"]
    FGDRYFOOD_KG = config["Food"]["FGDryFood"]["kg_per_serving"]
    FGSNACKS_KG = config["Food"]["FGSnacks"]["kg_per_serving"]
    FGSTOREPREPARED_KG = config["Food"]["FGStorePrepared"]["kg_per_serving"]
    


In [14]:
lookup_df = pd.DataFrame({
    'Type': ['Meat & Fish', 'Dairy & Eggs', 'Fruits & Vegetables', 'Dry Foods', 'Baked Goods' ,
    'Snacks, Condiments, Oils & Other', 'Store-Prepared Items'],
    'Servings_to_Kg': [FGMEAT_KG, FGDAIRY_KG, FGVEGETABLE_KG, FGDRYFOOD_KG, FGBAKED_KG, FGSNACKS_KG, FGSTOREPREPARED_KG],
    "Color": ["#116A65", "#00a0e1", "#466eb4", "#e6a532", "#d7642c","#73B55B", "#D82E5E"]
})

status_colors = {
            "Inedible Parts": "#26547C",
            "Plate Waste": "#FFD166",
            "Spoiled Food": "#EF476F"
        }
    
#color_mapping = dict(zip(lookup_df['Type'], lookup_df['Color']))

In [15]:
def load_data() -> dict[str, dd.DataFrame]:
    # Dictionary to store DataFrames by main folder and CSV file type
    dataframes = {}

    # List of the main folders to process
    main_folders = [f for f in os.listdir(PATH) if os.path.isdir(os.path.join(PATH, f))]
    df_dict = dict()

    # Iterate over each main folder
    for main_folder in main_folders:
        folder_path = os.path.join(PATH, main_folder) # type: ignore
        # Iterate over each run folder inside the main folder
        for run_folder in os.listdir(folder_path):
            run_path = os.path.join(folder_path, run_folder)
            if os.path.isdir(run_path):
                run_id = run_folder.split('_')[-1]  # Extract the run ID from the folder name
                # Read each CSV type and append the DataFrame to the respective list
                file_names = [f for f in os.listdir(os.path.join(folder_path, run_folder)) if os.path.isfile(os.path.join(folder_path,run_folder, f))]
                for file in file_names:
                    file_path = os.path.normpath(os.path.join(run_path, file))
                    if os.path.exists(file_path):
                        # Read the CSV file into a Dask DataFrame
                        df = dd.read_csv(file_path)
                        df_dict.setdefault(file[:-4], []).append(df)
        # Combine all runs into a single DataFrame for each CSV type
        combined_dfs = {csv_type: dd.concat(df_list, axis=0, ignore_index=True) 
                        for csv_type, df_list in df_dict.items()}
        
        # Store the combined DataFrames in the main dictionary under the main folder key
        dataframes[main_folder] = combined_dfs

    return dataframes

In [16]:
data = load_data()

ImportError: An error occurred while calling the read_csv method registered to the pandas backend.
Original Message: An error occurred while calling the read_csv method registered to the pandas backend.
Original Message: pyarrow>=10.0.1 is required for PyArrow backed StringArray.